In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
spark.conf.set("fs.azure.account.auth.type.adlsstorage4298.dfs.core.windows.net","SharedKey")
spark.conf.set("fs.azure.account.key.adlsstorage4298.dfs.core.windows.net","aWexrLVdOokH3MYN416YGi/feNTsJHD8GQjHipz74u4BVJMw108gRt+j1/wzD7LMOpP1z8/uGi36+AStkSWF0w==")

In [0]:
from pyspark.sql.types import *
schemaTransData = StructType([
    StructField("txnid", IntegerType(), True),
    StructField("txndate", StringType(), True),
    StructField("custid", IntegerType(), True),
    StructField("amount", DoubleType(), True),
    StructField("category", StringType(), True),
    StructField("subcategory", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("txntype", StringType(), True),
                    ])

In [0]:
transDataDF = spark.read.option("header",False).schema(schemaTransData).option("delimiter",",").csv("abfss://adlsdata@adlsstorage4298.dfs.core.windows.net/transactions/txns")

In [0]:
transDataDF.registerTempTable("transactions")

In [0]:
#1. Find the total_revenue based on category

In [0]:

%sql
select category,sum(amount) as total_revenue from transactions group by category;


category,total_revenue
Gymnastics,6617582.299999996
Winter Sports,6628206.3100000005
Jumping,4107379.3399999943
Team Sports,1.2405851640000027E7
Air Sports,2080245.0900000012
Indoor Games,5774627.899999999
Games,7417431.410000007
Outdoor Play Equipment,5765130.970000002
Water Sports,1.0748438760000013E7
Puzzles,1227776.4400000004


In [0]:
#2. Find the total number of transactions done by card

In [0]:
%sql
select count(txntype) as total_card_txns from transactions where txntype='credit';

total_card_txns
863585


In [0]:
#3. Find the highest selling category

In [0]:
%sql
select category, sum(amount) as Highest_selling from transactions group by category order by Highest_Selling desc limit 1;

category,Highest_selling
Outdoor Recreation,1.736480985999997E7


In [0]:
#4.Find the lowest selling subcategory in highest selling category

In [0]:
%sql
select subcategory, sum(amount) as Lowest_Selling from transactions where category='Outdoor Recreation' group by subcategory order by Lowest_Selling limit 1;

subcategory,Lowest_Selling
Equestrian,806128.8100000003


In [0]:
from pyspark.sql.types import *
schemaCustData = StructType([
    StructField("custid", IntegerType(), True),
    StructField("fname", StringType(), True),
    StructField("lname", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("profession", StringType(), True)
                    ])

In [0]:
custDataDF = spark.read.option("header",False).schema(schemaCustData).option("delimiter",",").csv("abfss://adlsdata@adlsstorage4298.dfs.core.windows.net/custdata/custs")

In [0]:
custDataDF.show(5)

+-------+--------+-------+---+--------------------+
 custid| fname| lname|age| profession|
+-------+--------+-------+---+--------------------+
4000001|Kristina| Chung| 55| Pilot|
4000002| Paige| Chen| 74| Teacher|
4000003| Sherri| Melton| 34| Firefighter|
4000004|Gretchen| Hill| 66|Computer hardware...|
4000005| Karen|Puckett| 74| Lawyer|
+-------+--------+-------+---+--------------------+
only showing top 5 rows

In [0]:
custDataDF.registerTempTable("customers")

In [0]:
#5.Find the total number of transactions done by customers with profession = Carpenter

In [0]:
%sql
select t.custid, count(t.txnid) as Total_Transactions from transactions t JOIN customers c ON t.custid = c.custid WHERE c.profession = 'Carpenter' GROUP BY t.custid;

custid,Total_Transactions
4005905,125
4001895,102
4001600,108
4007179,100
4001892,108
4008152,97
4008195,94
4004541,90
4007986,98
4006297,99
